In [45]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [46]:
# Open file
f = open('uspop10.asc','r')

In [47]:
# Load in dataframe
linesdf = []
for line in f:
    linesdf.append(line.split(" "))
df = pd.DataFrame(linesdf)

In [48]:
# Clean dataframe

# Get useful values belonging to the dataframe
ncols = int(df[9][0][:-1])
nrows = int(df[9][1][:-1])
cellsize = float(df[6][4][:-1])

x_left = float(df[5][2][:-1])
y_down = float(df[5][3][:-1])
x_right = x_left + ncols*cellsize
y_up = y_down + nrows*cellsize

upper_west_point = (x_left, y_up)
lower_east_point = (x_right, y_down)

# Now we get rid of unuseful rows and columns
df = df.drop([ncols],axis=1)
for i in range(6):
    df = df.drop(df.index[0])

# Next, we assign the correct values to both latitude and longitude axis
indexes_x=[]
for i in np.arange(x_left, x_right, cellsize):
    indexes_x.append(i)
indexes_y=[]
for j in np.arange(y_up, y_down, -cellsize):
    indexes_y.append(j)
    
df.columns = indexes_x
df.index = indexes_y
df = df.replace('-9999', 0)
df = df.astype(float)

#Useful data display
print("Number of columns is:", ncols)
print("Number of rows is:", nrows)
print("Latitude and longitude stride between cells is:", cellsize)
print("Upper West coordinates of dataframe are:", upper_west_point)
print("Lower East coordinates of dataframe are:", lower_east_point)

Number of columns is: 6940
Number of rows is: 2984
Latitude and longitude stride between cells is: 0.0083333333333334
Upper West coordinates of dataframe are: (-124.77500000221, 49.38333333575386)
Lower East coordinates of dataframe are: (-66.94166666887622, 24.516666669087)


In [49]:
test_tower_Lat = 40.711012 + 2*cellsize 
test_tower_Lng = -74.013064 + 2*cellsize

# 100x140 (LatxLng) dimensions of image grid and popullation grid of dataset

#Hardcoded starting points based on geographical diversity and number of images
start_Lat = 38.633333335754472 # df.index[1290]
start_Lng = -77.858333335506003 # df.columns[5630]

end_Lat, end_Lng = start_Lat + 100*cellsize*2, start_Lng + 70*cellsize*2
print(start_Lat, start_Lng, end_Lat, end_Lng)

38.63333333575447 -77.858333335506 40.30000000242115 -76.69166666883933


In [50]:
df_filtered = df.iloc[1190:1190+100,5630:5630+140] # Change this when the new dataframe is ready
np.sum(df_filtered.values)

5626147.8268373506

In [51]:
import urllib.request
from PIL import Image
import os
import math

class GoogleMapDownloader:
    """
        A class which generates high resolution google maps images given
        a longitude, latitude and zoom level
    """

    def __init__(self, lat, lng, zoom=12):
        """
            GoogleMapDownloader Constructor

            Args:
                lat:    The latitude of the location required
                lng:    The longitude of the location required
                zoom:   The zoom level of the location required, ranges from 0 - 23
                        defaults to 12
        """
        self._lat = lat
        self._lng = lng
        self._zoom = zoom

    def getXY(self):
        """
            Generates an X,Y tile coordinate based on the latitude, longitude 
            and zoom level

            Returns:    An X,Y tile coordinate
        """
        
        tile_size = 256

        # Use a left shift to get the power of 2
        # i.e. a zoom level of 2 will have 2^2 = 4 tiles
        numTiles = 1 << self._zoom

        # Find the x_point given the longitude
        point_x = (tile_size/ 2 + self._lng * tile_size / 360.0) * numTiles // tile_size

        # Convert the latitude to radians and take the sine
        sin_y = math.sin(self._lat * (math.pi / 180.0))

        # Calulate the y coorindate
        point_y = ((tile_size / 2) + 0.5 * math.log((1+sin_y)/(1-sin_y)) * -(tile_size / (2 * math.pi))) * numTiles // tile_size

        return int(point_x), int(point_y)

    def generateImage(self, **kwargs):
        """
            Generates an image by stitching a number of google map tiles together.
            
            Args:
                start_x:        The top-left x-tile coordinate
                start_y:        The top-left y-tile coordinate
                tile_width:     The number of tiles wide the image should be -
                                defaults to 5
                tile_height:    The number of tiles high the image should be -
                                defaults to 5
            Returns:
                A high-resolution Goole Map image.
        """

        start_x = kwargs.get('start_x', None)
        start_y = kwargs.get('start_y', None)
        tile_width = kwargs.get('tile_width', 4)
        tile_height = kwargs.get('tile_height', 4)

        # Check that we have x and y tile coordinates
        if start_x == None or start_y == None :
            start_x, start_y = self.getXY()

        # Determine the size of the image
        width, height = 256 * tile_width, 256 * tile_height

        #Create a new image of the size require
        map_img = Image.new('RGB', (width,height))

        for x in range(0, tile_width):
            for y in range(0, tile_height) :
                url = 'https://mt0.google.com/vt/lyrs=s&x='+str(start_x+x)+'&y='+str(start_y-y)+'&z='+str(self._zoom)
                # Where lyrs=
                # h = roads only
                # m = standard roadmap
                # p = terrain
                # r = somehow altered roadmap
                # s = satellite only
                # t = terrain only
                # y = hybrid
                # Also, we can add &scale=2 at the end to increase pixels to 512x512
                #print(url)
                current_tile = str(x)+'-'+str(y)
                urllib.request.urlretrieve(url, current_tile)
            
                im = Image.open(current_tile)
                map_img.paste(im, (x*256, (3-y)*256))
              
                os.remove(current_tile)

        return map_img

In [52]:
def main():
    # Create a new instance of GoogleMap Downloader
    fail_count = 0
    for Lat in range(5): # Set back to 100
        for Lng in range(70): # Set back to 70
            gmd = GoogleMapDownloader(start_Lat + (Lat*cellsize*2), start_Lng + (Lng*cellsize*2), 16)
            #gmd = GoogleMapDownloader(40.711012, -74.013064, 16)
            #print("The tile coorindates are {}".format(gmd.getXY()))

            try:
                # Get the high resolution image
                img = gmd.generateImage()
                #img = img.resize((200, 200), Image.ANTIALIAS)
            except IOError:
                fail_count += 1
                print(fail_count,": Could not generate the image at Lat:", Lat, "and Lng:", Lng)
            else:
                #Save the image to disk
                img.save(str(Lat)+"_"+str(Lng)+".png")
                #print("The map has successfully been created in the loop: " + str(i))

if __name__ == '__main__':  main()

1 : Could not generate the image at Lat: 1 and Lng: 7
2 : Could not generate the image at Lat: 1 and Lng: 15
3 : Could not generate the image at Lat: 3 and Lng: 51
4 : Could not generate the image at Lat: 4 and Lng: 7
